In [7]:
import torch as t
from torch.utils.data import DataLoader
import torchvision as tv

In [5]:
transform = tv.transforms.Compose([tv.transforms.ToTensor(),
                                  tv.transforms.Normalize((0.5,), (0.5,)),
                             ])

In [8]:
train_ts = tv.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ts = tv.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_dl = DataLoader(train_ts, batch_size=32, shuffle=True, drop_last=False)
test_dl = DataLoader(test_ts, batch_size=64, shuffle=True, drop_last=False)

In [9]:
#自定义网络结构的实现方法
class CNN_Net(t.nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.cnn_layers = t.nn.Sequential(
            t.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1, stride=1),
            t.nn.MaxPool2d(kernel_size=2, stride=2),
            t.nn.ReLU(),
            t.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, padding=1, stride=1),
            t.nn.MaxPool2d(kernel_size=2, stride=2),
            t.nn.ReLU(),
        )
        self.fc_layers = t.nn.Sequential(
            t.nn.Linear(7 * 7 * 32, 200),
            t.nn.ReLU(),
            t.nn.Linear(200, 100),
            t.nn.ReLU(),
            t.nn.Linear(100, 10),
            t.nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        out = self.cnn_layers(x)
        out = out.view(-1, 7 * 7 * 32)
        out = self.fc_layers(out)
        return out

In [11]:
#训练
#model = CNN_Net().cuda()
model = CNN_Net()
loss_fn= t.nn.CrossEntropyLoss()#交叉熵损失函数
optimizer = t.optim.Adam(model.parameters(), lr=1e-3)#adam优化器

for s in range(5):
    print("run in step : %d"%s)
    for i, (x_train, y_train) in enumerate(train_dl):
        #x_train = x_train.cuda()
        #y_train = y_train.cuda()
        y_pred = model.forward(x_train)
        train_loss = loss_fn(y_pred, y_train)
        if (i + 1) % 100 == 0:
            print(i + 1, train_loss.item())
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

run in step : 0
100 0.4613865911960602
200 0.4756956100463867
300 0.05229443684220314
400 0.15475468337535858
500 0.13041836023330688
600 0.06947698444128036
700 0.13500292599201202
800 0.06984113901853561
900 0.06036350503563881
1000 0.033883947879076004
1100 0.02862543612718582
1200 0.010611496865749359
1300 0.015622371807694435
1400 0.11151580512523651
1500 0.04479527473449707
1600 0.02578495256602764
1700 0.015884483233094215
1800 0.004390612244606018
run in step : 1
100 0.07802199572324753
200 0.02140556462109089
300 0.07667724043130875
400 0.037296004593372345
500 0.23584680259227753
600 0.04446800798177719
700 0.029841862618923187
800 0.057519782334566116
900 0.0244471225887537
1000 0.0971626415848732
1100 0.0013018350582569838
1200 0.1106468141078949
1300 0.005024254787713289
1400 0.010726489126682281
1500 0.003038842463865876
1600 0.007501841057091951
1700 0.0054342783987522125
1800 0.0032863905653357506
run in step : 2
100 0.16931504011154175
200 0.1293685883283615
300 0.0187

In [15]:
#预测
total = 0;
correct_count = 0

model.eval()
for test_images, test_labels in test_dl:
    with t.no_grad():
        #pred_labels = model(test_images.cuda())
        pred_labels = model(test_images)
    predicted = t.max(pred_labels, 1)[1]
    #correct_count += (predicted == test_labels.cuda()).sum()
    correct_count += (predicted == test_labels).sum()
    total += len(test_labels)
print(correct_count.cpu().detach().numpy(), total)

print("total acc : %.4f\n"%(correct_count.cpu().detach().numpy() / total))
t.save(model, './cnn_mnist_model.pt')

9903 10000
total acc : 0.9903



D:\Anaconda3\envs\torch\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN_Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [27]:
#pt模型转为onnx格式
#dummy_input = torch.randn(1, 1, 28, 28, device='cuda')
dummy_input = t.randn(1, 1, 28, 28, device='cpu')
model = t.load("./cnn_mnist_model.pt")
t.onnx.export(model, dummy_input, "cnn_mnist.onnx", verbose=True)

graph(%input.1 : Float(1, 1, 28, 28),
      %cnn_layers.0.weight : Float(8, 1, 3, 3),
      %cnn_layers.0.bias : Float(8),
      %cnn_layers.3.weight : Float(32, 8, 3, 3),
      %cnn_layers.3.bias : Float(32),
      %fc_layers.0.weight : Float(200, 1568),
      %fc_layers.0.bias : Float(200),
      %fc_layers.2.weight : Float(100, 200),
      %fc_layers.2.bias : Float(100),
      %fc_layers.4.weight : Float(10, 100),
      %fc_layers.4.bias : Float(10)):
  %11 : Float(1, 8, 28, 28) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input.1, %cnn_layers.0.weight, %cnn_layers.0.bias) # D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\modules\conv.py:350:0
  %12 : Float(1, 8, 14, 14) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%11) # D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:539:0
  %13 : Float(1, 8, 14, 14) = onnx::Relu(%12) # D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py

In [ ]:
import cv2 as cv
import numpy as np

mnist_net = cv.dnn.readNetFromONNX("cnn_mnist.onnx")
image = cv.imread("test_image.jpg")
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
cv.imshow("input", gray)
blob = cv.dnn.blobFromImage(gray, 0.00392, (28, 28), (0.5)) / 0.5
print(blob.shape)
mnist_net.setInput(blob)
result = mnist_net.forward()
pred_label = np.argmax(result, 1)
print("predit label : %d"%pred_label)
cv.waitKey(0)
cv.destroyAllWindows()

(1, 1, 28, 28)
predit label : 0
